<a href="https://colab.research.google.com/github/deveshsangwan/Skin-lesions-classification/blob/main/Xception_Skin_lesions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
import os
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
y_train = np.load("/content/drive/My Drive/Datasets/Skin_npy/y_train.npy")

In [6]:
X_train = np.load("/content/drive/My Drive/Datasets/Skin_npy/x_train.npy")

In [7]:
X_val = np.load("/content/drive/My Drive/Datasets/Skin_npy/x_validate.npy")

In [8]:
y_val = np.load("/content/drive/My Drive/Datasets/Skin_npy/y_validate.npy")

In [9]:
X_train.shape, X_val.shape

((7210, 192, 256, 3), (802, 192, 256, 3))

In [10]:
y_train.shape, y_val.shape

((7210, 7), (802, 7))

In [11]:

pre_trained_model = Xception(input_shape=(192, 256, 3), include_top=False, weights="imagenet")

83689472/83683744 [==============================] - 2s 0us/step


In [12]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False
    
print(len(pre_trained_model.layers))

input_1
block1_conv1
block1_conv1_bn
block1_conv1_act
block1_conv2
block1_conv2_bn
block1_conv2_act
block2_sepconv1
block2_sepconv1_bn
block2_sepconv2_act
block2_sepconv2
block2_sepconv2_bn
conv2d
block2_pool
batch_normalization
add
block3_sepconv1_act
block3_sepconv1
block3_sepconv1_bn
block3_sepconv2_act
block3_sepconv2
block3_sepconv2_bn
conv2d_1
block3_pool
batch_normalization_1
add_1
block4_sepconv1_act
block4_sepconv1
block4_sepconv1_bn
block4_sepconv2_act
block4_sepconv2
block4_sepconv2_bn
conv2d_2
block4_pool
batch_normalization_2
add_2
block5_sepconv1_act
block5_sepconv1
block5_sepconv1_bn
block5_sepconv2_act
block5_sepconv2
block5_sepconv2_bn
block5_sepconv3_act
block5_sepconv3
block5_sepconv3_bn
add_3
block6_sepconv1_act
block6_sepconv1
block6_sepconv1_bn
block6_sepconv2_act
block6_sepconv2
block6_sepconv2_bn
block6_sepconv3_act
block6_sepconv3
block6_sepconv3_bn
add_4
block7_sepconv1_act
block7_sepconv1
block7_sepconv1_bn
block7_sepconv2_act
block7_sepconv2
block7_sepconv2_

In [13]:
last_layer = pre_trained_model.get_layer('block14_sepconv2_act')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 6, 8, 2048)


In [14]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [15]:
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        shear_range = 10)

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [16]:
for layer in pre_trained_model.layers:
    layer.trainable = True

In [17]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [18]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [19]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 95, 127, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 95, 127, 32)  128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 95, 127, 32)  0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [20]:
batch_size = 64
epochs = 35
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/35
112/112 [==============================] - 271s 2s/step - loss: 0.9521 - acc: 0.6836 - val_loss: 1.2769 - val_acc: 0.7005
Epoch 2/35
112/112 [==============================] - 267s 2s/step - loss: 0.6652 - acc: 0.7702 - val_loss: 0.6571 - val_acc: 0.7760
Epoch 3/35
112/112 [==============================] - 266s 2s/step - loss: 0.5372 - acc: 0.8142 - val_loss: 0.5782 - val_acc: 0.7917
Epoch 4/35
112/112 [==============================] - 267s 2s/step - loss: 0.4482 - acc: 0.8382 - val_loss: 0.5153 - val_acc: 0.8177
Epoch 5/35
112/112 [==============================] - 268s 2s/step - loss: 0.3819 - acc: 0.8636 - val_loss: 0.5535 - val_acc: 0.7812
Epoch 6/35
112/112 [==============================] - 269s 2s/step - loss: 0.3330 - acc: 0.8832 - val_loss: 0.5174 - val_acc: 0.8021
Epoch 7/35
112/112 [==============================] - 269s 2s/step - loss: 0.2852 - acc: 0.8948 - val_loss: 0.4189 - val_acc: 

In [21]:
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

26/26 [==============================] - 8s 303ms/step - loss: 0.7449 - acc: 0.8616
Validation: accuracy = 0.861596  ;  loss_v = 0.744947


In [23]:
X_test = np.load("/content/drive/My Drive/Datasets/Skin_npy/x_test.npy")

In [24]:
y_test = np.load("/content/drive/My Drive/Datasets/Skin_npy/y_test.npy")

In [25]:
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

63/63 [==============================] - 20s 315ms/step - loss: 0.8008 - acc: 0.8457
Test: accuracy = 0.845731  ;  loss = 0.800797


In [26]:
model.save("/content/drive/My Drive/Xception.h5")